In [1]:
cost = 3000
if cost > 2000:
  cost = cost * 0.95
  print('cost',cost)

cost 2850.0


In [6]:
import pandas as pd

# 讀取 Excel 檔案（請修改 "your_file.xlsx" 為實際檔案名稱）
df = pd.read_excel("復亨銷售紀錄-20250303.xlsx")

# 確保 '交易日期' 欄位為 datetime 格式
df["銷貨日期"] = pd.to_datetime(df["銷貨日期"])

# 篩選交易日期大於 2021 年（即 2022-01-01 之後）
filtered_df = df[(df["銷貨日期"] < "2021-12-31") & (df["銷貨日期"] > "2020-12-31")]

# 取得不重複的顧客代號
distinct_customers = filtered_df["客戶代號"].nunique()

# 顯示結果
print(distinct_customers)






1257


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# 指定 Excel 檔案路徑
file_paths = [
    "2020~2021_label.xlsx",
    "2021~2022_label.xlsx",
    "2022~2023_label.xlsx"
]

# 讀取所有 Excel，忽略標題（header=None）
df_list = [pd.read_excel(file, skiprows=1, header=None) for file in file_paths]

# 縱向合併（忽略欄位名稱）
df_combined = pd.concat(df_list, ignore_index=True)

test_data = pd.read_excel("2023~2024_label.xlsx")

# 假設最後一欄是標籤 (Label)，其餘為特徵 (Features)
X_train = df_combined.iloc[:,  -2:]  # 取所有列，除了最後一列
# y_train = train_data['布林值']   # 取最後一列作為標籤
y_train = df_combined.iloc[:,  -3:-2]   # 取最後一列作為標籤

X_test = test_data.iloc[:, -2:]
# y_test = test_data['布林值']
y_test = test_data.iloc[:,  -3:-2]

X_train

,7,8
0,4,12600
1,3,8565
2,1,3400
3,5,7860
4,2,2610
...,...,...
5029,2,1620
5030,4,2180
5031,2,2560
5032,4,3045


In [9]:
# 標準化特徵值
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# 建立 SVM 模型
svm_model = SVC(kernel='poly', C=0.7, gamma='scale')
svm_model.fit(X_train, y_train)

# 預測測試集
y_pred = svm_model.predict(X_test)

# 計算準確率與分類報告
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"測試準確率: {accuracy:.4f}")
print("分類報告:")
print(report)


c:\Users\LG\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


測試準確率: 0.5535
分類報告:
              precision    recall  f1-score   support

           0       0.55      1.00      0.71       602
           1       0.40      0.01      0.01       318
           2       1.00      0.01      0.02       173

    accuracy                           0.55      1093
   macro avg       0.65      0.34      0.25      1093
weighted avg       0.58      0.55      0.40      1093



c:\Users\LG\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [11]:
# 修正 test_data 索引篩選的錯誤邏輯
test_data = pd.DataFrame(X_test)  # 假設 test_data 來自 X_test
test_data['actual_label'] = y_test
test_data['predicted_label'] = y_pred

# 過濾出預測錯誤的樣本
errors = test_data[test_data['actual_label'] == test_data['predicted_label']]

print("預測錯誤的樣本:")
print(errors)

預測錯誤的樣本:
      2023年總次數  2023年總金額  actual_label  predicted_label
4            6      8068             0                0
8            5      4318             0                0
9            2      1260             0                0
13           9      7118             0                0
17           4      1985             0                0
...        ...       ...           ...              ...
1071         2      2180             0                0
1072         2      1260             0                0
1074         6      7350             0                0
1075         5      3695             0                0
1090         3      2520             0                0

[605 rows x 4 columns]


In [12]:
y_test

,布林值
0,2
1,1
2,1
3,2
4,0
...,...
1088,1
1089,1
1090,0
1091,1
